In [1]:
import pandas as pd

In [2]:
try: # Running Locally, yakinkan "file_" berada di folder "data"
    mengajar = pd.read_csv('data/mengajar.csv', encoding='utf8')
    dosen = pd.read_csv('data/tb_dosen.csv', encoding='utf8')
    kelas = pd.read_csv('data/tb_kelas.csv', encoding='utf8')
    krs = pd.read_csv('data/tb_krs.csv', encoding='utf8')
    mahasiswa = pd.read_csv('data/tb_mahasiswa.csv', encoding='utf8')
    kuliah = pd.read_csv('data/tb_mata_kuliah.csv', encoding='utf8')
    nilai = pd.read_csv('data/tb_nilai.csv', encoding='utf8')
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/mengajar.csv
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/tb_dosen.csv
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/tb_kelas.csv
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/tb_krs.csv
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/tb_mahasiswa.csv
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/tb_mata_kuliah.csv
    !wget -P data/ https://raw.githubusercontent.com/taufiksutanto/cs02-eda/main/data/tb_nilai.csv
    mengajar = pd.read_csv('data/mengajar.csv', encoding='utf8')
    dosen = pd.read_csv('data/tb_dosen.csv', encoding='utf8')
    kelas = pd.read_csv('data/tb_kelas.csv', encoding='utf8')
    krs = pd.read_csv('data/tb_krs.csv', encoding='utf8')
    mahasiswa = pd.read_csv('data/tb_mahasiswa.csv', encoding='utf8')
    kuliah = pd.read_csv('data/tb_mata_kuliah.csv', encoding='utf8')
    nilai = pd.read_csv('data/tb_nilai.csv', encoding='utf8')
    
mengajar.shape, dosen.shape, kelas.shape, krs.shape, mahasiswa.shape, kuliah.shape, nilai.shape

((14, 4), (4, 6), (4, 4), (7, 6), (15, 8), (9, 3), (33, 7))

Permasalahan:

**1. Apakah ada dosen yang cenderung memberikan nilai cenderung lebih tinggi (atau lebih rendah) dibandingkan dosen lainnya?**

In [3]:
nilai_dosen = nilai.groupby([nilai['NIDN']]).mean()[['nilai']]
nilai_dosen = nilai_dosen.rename({'nilai': 'mean_nilai'}, axis=1)
nilai_dosen = pd.merge(dosen, nilai_dosen, left_on='NIDN', right_index=True)[['NIDN', 'nama', 'mean_nilai']]
nilai_dosen = nilai_dosen.set_index(['NIDN'])
std = nilai.groupby([nilai['NIDN']]).std()[['nilai']]
std = std.rename({'nilai': 'std_nilai'}, axis=1)
nilai_dosen = pd.concat([nilai_dosen, std], axis=1)
nilai_dosen

,nama,mean_nilai,std_nilai
NIDN,,,
123,Andri Basuki,85.750000,7.741355
234,zulfikar,80.428571,15.883504
345,Annisa Fitria,60.090909,16.220077
787,yudi,70.571429,13.010984


Interpretasi Nomor 1 :

Terdapat kecenderungan dalam pemberian nilai, ada dosen yang cenderung memberikan nilai lebih tinggi dibandingkan dosen lainnya, dan ada pula dosen yang cenderung memberikan nilai lebih rendah dibandingkan dosen lainnya.

Andri Basuki memberikan nilai yang lebih tinggi dibanding dosen lainnya dilihat dari mean nilai yang tinggi dengan standar deviasi nilai yang rendah dan Annisa Fitria memberikan nilai lebih rendah dibandingkan dosen lainnya dilihat dari mean nilai yang rendah dengan standar deviasi nilai yang tinggi.

Namun, dalam hal ini masih perlu tambahan data yaitu data mengenai jumlah kelas yang diajarkan sehingga dapat mempertimbangkan mean nilai dari setiap dosen. Karena akan terjadi ketidakadilan jika jumlah kelasnya berbeda.

**2. Menurut data yang ada, apakah terdapat perbedaan dalam pemberian nilai antara dosen dengan Pendidikan Terakhir S3 dan S2?**

In [4]:
dosen = dosen.set_index(['NIDN'])
sarjana = pd.merge(dosen, nilai_dosen, left_on='NIDN', right_index=True)
sarjana[['nama_x', 'Pendidikan_Terakhir', 'mean_nilai', 'std_nilai']]

,nama_x,Pendidikan_Terakhir,mean_nilai,std_nilai
NIDN,,,,
123,Andri Basuki,S3,85.750000,7.741355
234,zulfikar,S3,80.428571,15.883504
787,yudi,S2,70.571429,13.010984
345,Annisa Fitria,S2,60.090909,16.220077


Interpretasi Nomor 2 :

Jika hanya dilihat dari mean nilai, terdapat perbedaan dalam pemberian nilai dari dosen berdasarkan pendidikan terakhirnya. Dosen dengan pendidikan terakhir S3 cenderung memberikan nilai yang lebih tinggi dibandingkan dengan dosen dengan pendidikan terakhir S2.

Namun, jika dilihat dari standar deviasi (secara perorangan), perbedaan pemberian nilai dari dosen berdasrkan pendidikan terakhirnya tidak terlalu jauh berbeda. Karena Andri Basuki yang memiliki rata-rata nilai tinggi, standar deviasi rendah, dapat tergolong dosen yang memberikan nilai tinggi, pendidikan terakhir beliau adalah S3. Sedangkan Annisa Fitria yang memberikan nilai rendah dengan standar deviasi yang tinggi memiliki pendidikan terahir S2. Hal ini bukan berarti pendidikan terakhir dosen mempengaruhi nilai karena ada Zulfikar yang memberikan nilai tinggi dan standar deviasi yang tinggi pula itu memiliki pendidikan terakhir S3 sama seperti Andri Basuki. 

**3. Apakah terdapat perbedaan performa akademik antara mahasiswa yang berasal dari Madrasah/SMK dan SMU?**

In [5]:
asal_sekolah = pd.merge(mahasiswa, nilai, on='nim')
asal_sekolah = asal_sekolah[['Asal_Sekolah', 'id_mata_kuliah', 'nilai', 'Asal_Daerah']]
std = asal_sekolah.groupby([asal_sekolah[ 'Asal_Sekolah' ], asal_sekolah['Asal_Daerah'], asal_sekolah[ 'id_mata_kuliah' ]]).std()[['nilai']]
rata_nilai = asal_sekolah.groupby([asal_sekolah['Asal_Sekolah'], asal_sekolah['Asal_Daerah'], asal_sekolah['id_mata_kuliah']]).mean()[['nilai']]
std = std.rename({'nilai': 'std_nilai'}, axis=1)
rata_nilai = rata_nilai.rename({'nilai': 'rata2_nilai'}, axis=1)
rata_nilai

rata2_nilai
Asal_Sekolah Asal_Daerah id_mata_kuliah             
MA           Lokal       CS01                   90.0
                         KAL01                  71.0
                         STA02                  31.0
             Luar Daerah CS01                   90.0
                         KAL01                  92.0
                         KAL02                  65.0
                         STA02                  56.0
SMK          Lokal       CS01                   88.5
                         KAL01                  90.5
                         STA02                  88.0
             Luar Daerah CS01                   55.0
                         KAL01                  85.0
                         KAL02                  74.0
                         STA02                  70.0
SMU          Lokal       CS01                   60.0
                         KAL01                  70.0
                         KAL02                  55.0
                         STA02                  72.0
             Luar Daerah CS01                   91.0
                         KAL01                  72.0
                         KAL02                  60.0
                         STA02                  45.0

Interpretasi Nomor 3 :

Terdapat perbedaan performa akademik di antara mahasiswa yang berasal dari MA/SMK dan SMU. Secara umum diketahui bahwa MA atau Madrasah Aliyah lebih menekankan pelajaran yang berhubungan dengan keagaamaan, SMK menekankan pelajaran kejuruan sesuai jurusan yang dipilih, kemudian untuk SMU menekankan pelajaran umum. Selain itu, ada beberapa sekolahan yang berasal dari daerah lokal maupun luar daerah yang sudah modern, dimana mereka mengikuti kurikulum terbaru, namun untuk daerah terpencil terkadang mereka terlambat untuk mengikuti kurikulum terbaru. 

Namun, hal tersebut tidak mempengaruhi pemahaman dan penguasaan mereka terhadap dasar pengetahuan dari mata kuliah yang diberikan. Hal ini terlihat dari rata-rata MA luar daerah > MA lokal, SMK luar daerah < SMK lokal, SMU luar daerah > SMU lokal.

Performa akademik yang ditunjukkan oleh hasil analisis data menunjukkan bahwa pada mata kuliah STA02 sangat terlihat perbedaan antara mahasiswa yang berasal dari MA dan SMU luar daerah dengan SMK dan SMU lokal memiliki perbedaan nilai yang cukup jauh. Mahasiswa yang berasal dari SMK cenderung memiliki nilai STA yang tinggi sedangkan yang berasal dari MA dan SMU luar daerah memiliki nilai yang cenderung rendah. Pada mata kuliah CS01 mahasiswa yang berasal dari SMK lokal dan SMU lokal memiliki nilai yang cenderung rendah dibandingkan yang lainnya. Pada mata kuliah KAL02 mahasiswa yang berasal dari MA lokal SMK lokal tidak mengambil mata kuliah tersebut. Hal ini mungkin terjadi karena data yang diberikan kurang lengkap padahal nilai KAL01 mereka cukup bagus.
Dalam hal ini dapat mempengaruhi performa akademik antara mahasiswa akan berbeda karena tidak semuanya mengambil mata kuliah yang sama.

Permasalahan:

**4. Apakah ada perbedaan yang nampak di data terhadap kemampuan akademik antara mahasiswa dan mahasiswi?**

In [6]:
mahasiswa = mahasiswa[['nim', 'nama', 'Gender',]]
nilai = nilai[['nim','nilai','id_mata_kuliah']]
mahasiswa = mahasiswa.set_index(['nim'])
nilai = nilai.set_index(['nim'])
nilai_gender= pd.merge(mahasiswa, nilai, left_on='nim', right_index=True)
nilai_gender = pd.merge(kuliah,nilai_gender,  on='id_mata_kuliah')
nilai_gender1 = nilai_gender[['nama','nama_mata_kuliah','SKS','Gender','nilai']] 
nilai_gender1


,nama,nama_mata_kuliah,SKS,Gender,nilai
0,Muhammad Amir,Kalkulus Dasar,3,Pria,70
1,Samsul Kurnia,Kalkulus Dasar,3,Pria,80
2,Bambang Arifin,Kalkulus Dasar,3,Pria,85
3,Udin,Kalkulus Dasar,3,Pria,90
4,Iwan,Kalkulus Dasar,3,Pria,92
5,Bambang Arifin,Kalkulus Dasar,3,Pria,91
6,Wati,Kalkulus Dasar,3,Wanita,93
7,Ani,Kalkulus Dasar,3,Wanita,85
8,Rahmawati,Kalkulus Dasar,3,Wanita,60
9,Asep,Kalkulus Dasar,3,Pria,63


In [7]:
std_nilai = nilai_gender1.groupby([nilai_gender1[ 'nama_mata_kuliah' ],nilai_gender1[ 'Gender' ]]).std()[['nilai']]
rata_nilai = nilai_gender1.groupby([nilai_gender1[ 'nama_mata_kuliah' ],nilai_gender1[ 'Gender' ]]).mean()[['nilai']]
std_nilai = std_nilai.rename({'nilai': 'std_nilai'}, axis=1)
rata_nilai = rata_nilai.rename({'nilai': 'rata2_nilai'}, axis=1)
rata_nilai

rata2_nilai
nama_mata_kuliah          Gender             
Kalkulus Dasar            Pria      81.222222
                          Wanita    74.833333
Kalkulus Lanjut           Pria      60.000000
                          Wanita    74.000000
Pendahuluan Ilmu Komputer Pria      91.000000
                          Wanita    72.500000
Statistika Matematika     Pria      48.666667
                          Wanita    65.250000

Interpretasi Nomor 4 :

Ya, terdapat perbedaan kemampuan akademik antara mahasiswa dan mahasiswi. Bisa kita lihat nilai antara mahasiswa dan mahasiswi. Berdasarkan rata-rata nilai secara keseluruhan, rata-rata nilai mahasiswa lebih tinggi dibanding mahasiswi dengan asumsi datanya terdistribusi normal. Namun, bila dilihat secara satu persatu, mata kuliah permata kuliah, ada beberapa mata kuliah yang rata-rata nilainya diungguli oleh mahasiswi, seperti kalkulus lanjut dan statistika matematika.

Jika dilihat dari standar deviasi nilai mahasiswa lebih kecil, sehingga nial bisa diasumsikan

Permasalahan:

**5. Apakah ada perbedaan prestasi akademik yang nampak di data antar angkatan mahasiswa?**

In [8]:
nilai_mahasiswai = pd.merge(nilai,mahasiswa,on='nim')
nilai_mahasiswaz = nilai_mahasiswai[['id_mata_kuliah','nilai']]
rata_nilai_mahasiswa = nilai_mahasiswaz.groupby([nilai_mahasiswaz[ 'id_mata_kuliah' ]]).mean()[['nilai']]
std_nilai_mahasiswa = nilai_mahasiswaz.groupby([nilai_mahasiswaz[ 'id_mata_kuliah' ]]).std()[['nilai']]

In [9]:
std_nilai

std_nilai
nama_mata_kuliah          Gender           
Kalkulus Dasar            Pria    10.986102
                          Wanita  15.380724
Kalkulus Lanjut           Pria     5.000000
                          Wanita        NaN
Pendahuluan Ilmu Komputer Pria     1.000000
                          Wanita  17.559423
Statistika Matematika     Pria     8.082904
                          Wanita  24.212600

In [10]:
rata_nilai

rata2_nilai
nama_mata_kuliah          Gender             
Kalkulus Dasar            Pria      81.222222
                          Wanita    74.833333
Kalkulus Lanjut           Pria      60.000000
                          Wanita    74.000000
Pendahuluan Ilmu Komputer Pria      91.000000
                          Wanita    72.500000
Statistika Matematika     Pria      48.666667
                          Wanita    65.250000

Interpretasi Nomor 5 :

Setelah mengelola data tersebut, bisa disimpulkan bahwa perbedaan prestasi akademik antara angkatan 2019 dan 2020 signifikan dibeberapa mata kuliah terutama pada mata kuliah KAL01 dengan selisih rata-rata tertinggi tetapi terdapat perbedaan yang tidak signifikan pada mata kuliah CS01.

Permasalahan:

**6. Apakah ada permasalahan di data atau informasi lain yang anda temukan di data?**

In [11]:
nilai_gender1.groupby (nilai_gender1 [ 'nama_mata_kuliah' ]).count()[['nama']]

,nama
nama_mata_kuliah,
Kalkulus Dasar,15
Kalkulus Lanjut,4
Pendahuluan Ilmu Komputer,7
Statistika Matematika,7


In [12]:
kuliah.head(100)

,id_mata_kuliah,nama_mata_kuliah,SKS
0,KAL01,Kalkulus Dasar,3
1,STA01,Statistika Elementer,3
2,MAT01,Matematika Dasar,3
3,STA02,Statistika Matematika,4
4,OR01,Operation Research,4
5,CS01,Pendahuluan Ilmu Komputer,3
6,MAT73,Aljabar Abstrak,4
7,MAT53,Aljabar linear,3
8,KAL02,Kalkulus Lanjut,4


Nomor 6

Kami menemukan permasalahan pada data di nomor 4 ketika digroupby dan std terdapat Nan di mata kuliah kalkulus lanjut dengan gender wanita.

Informasi lainnya : Pada tabel mata kuliah terdapat 9 mata kuliah yaitu Kalkulus dasar, Statistika Elementer, Matematika Dasar, Statistika Matematika, Operation Research, Pendahuluan Ilmu Komputer, Aljabar Abstrak, Aljabar linear, dan Kalkulus Lanjut, tetapi ternyata terdapat 5 mata kuliah yang sama sekali tidak diikuti oleh mahasiswa (tidak ada mahasiswa/mahasiswi yang mengikuti mata kuliah tsb), seperti Statistika Elementer, Matematika dasar, Operation Research, aljabar abstrak, aljabar linear.

#Kesimpulan

Dari data yang diberikan, kami dapat melihat perbedaan pemberian nilai antar dosen yang salah satu faktornya yaitu karena tingkat pendidikan terakhir dosen, kemudian terdapat perbedaan performa akademik mahasiswa berdasarkan asal sekolah (Madrasah/SMK/SMU) dan gender perbedaan prestasi akademik antar angkatan.

Dengan menggunakan data dosen dan nilai dapat terlihat kecenderungan dalam pemberian nilai, terdapat dosen yang cenderung memberikan nilai lebih tinggi/rendah dibandingkan dosen lainnya.

Andri Basuki memberikan nilai yang lebih tinggi dibanding dosen lainnya dilihat dari mean nilai yang tinggi dengan standar deviasi nilai yang rendah dan Annisa Fitria memberikan nilai lebih rendah dibandingkan dosen lainnya dilihat dari mean nilai yang rendah dengan standar deviasi nilai yang tinggi.

Namun, dalam hal ini terjadi ketidakakuratan dalam menghitung mean nilai karena jumlah kelas dari masing-masing berbeda, yaitu Andri Basuki 5 kelas, Zulfikar 2 kelas, Yudi 3 kelas, dan Annisa Fitria 4 kelas sehingga hal ini dapat mempengaruhi dalam menyimpulkan perbedaan pemberian nilai antara dosen.

Jika hanya dilihat dari mean nilai, terdapat perbedaan dalam pemberian nilai dari dosen berdasarkan pendidikan terakhirnya. Dosen dengan pendidikan terakhir S3 cenderung memberikan nilai yang lebih tinggi dibandingkan dengan dosen dengan pendidikan terakhir S2.

Namun, jika dilihat dari standar deviasi (secara perorangan), perbedaan pemberian nilai dari dosen berdasarkan pendidikan terakhirnya tidak terlalu jauh berbeda. Karena Andri Basuki yang memiliki rata-rata nilai tinggi, standar deviasi rendah, dapat tergolong dosen yang memberikan nilai tinggi, pendidikan terakhir beliau adalah S3. Sedangkan Annisa Fitria yang memberikan nilai rendah dengan standar deviasi yang tinggi memiliki pendidikan terahir S2. Hal ini bukan berarti pendidikan terakhir dosen mempengaruhi nilai karena ada Zulfikar yang memberikan nilai tinggi dan standar deviasi yang tinggi pula itu memiliki pendidikan terakhir S3 sama seperti Andri Basuki.

Dari data mahasiswa dan nilai kami dapat melihat perbedaan performa akademik di antara mahasiswa yang berasal dari MA/SMK dan SMU dan gender mahasiswa. 

Secara umum diketahui bahwa MA atau Madrasah Aliyah lebih menekankan pelajaran yang berhubungan dengan keagaamaan, SMK menekankan pelajaran kejuruan sesuai jurusan yang dipilih, kemudian untuk SMU menekankan pelajaran umum. Selain itu, ada beberapa sekolahan yang berasal dari daerah lokal maupun luar daerah yang sudah modern, dimana mereka mengikuti kurikulum terbaru, namun untuk daerah terpencil terkadang mereka terlambat untuk mengikuti kurikulum terbaru.

Namun, hal tersebut tidak mempengaruhi pemahaman dan penguasaan mereka terhadap dasar pengetahuan dari mata kuliah yang diberikan. Hal ini terlihat dari rata-rata MA luar daerah > MA lokal, SMK luar daerah < SMK lokal, SMU luar daerah > SMU lokal.

Performa akademik yang ditunjukkan oleh hasil analisis data menunjukkan bahwa pada mata kuliah STA02 sangat terlihat perbedaan antara mahasiswa yang berasal dari MA dan SMU luar daerah dengan SMK dan SMU lokal memiliki perbedaan nilai yang cukup jauh. Mahasiswa yang berasal dari SMK cenderung memiliki nilai STA yang tinggi sedangkan yang berasal dari MA dan SMU luar daerah memiliki nilai yang cenderung rendah. Pada mata kuliah CS01 mahasiswa yang berasal dari SMK lokal dan SMU lokal memiliki nilai yang cenderung rendah dibandingkan yang lainnya. Pada mata kuliah KAL02 mahasiswa yang berasal dari MA lokal SMK lokal tidak mengambil mata kuliah tersebut. Hal ini mungkin terjadi karena data yang diberikan kurang lengkap padahal nilai KAL01 mereka cukup bagus. Dalam hal ini dapat mempengaruhi performa akademik antara mahasiswa akan berbeda karena tidak semuanya mengambil mata kuliah yang sama.

Selain itu faktor asal sekolah,  perbedaan kemampuan akademik antara mahasiswa dan mahasiswi bisa dilihat dari gender. Berdasarkan rata-rata nilai secara keseluruhan, rata-rata nilai mahasiswa lebih tinggi dibanding mahasiswi dengan asumsi datanya terdistribusi normal. Namun, bila dilihat secara satu persatu, mata kuliah permata kuliah, ada beberapa mata kuliah yang rata-rata nilainya diungguli oleh mahasiswi, seperti kalkulus lanjut dan statistika matematika.

Jika dilihat dari standar deviasi nilai mahasiswa lebih kecil, sehingga nilai bisa diasumsikan.

Dalam data mahasiswa dan nilai pula, kami dapat melihat perbedaan prestasi akademik antara angkatan 2019 dan 2020 signifikan dibeberapa mata kuliah terutama pada mata kuliah KAL01 dengan selisih rata-rata tertinggi tetapi terdapat perbedaan yang tidak signifikan pada mata kuliah CS01

Setelah mengolah data tersebut, kami menemukan permasalahan dalam menentukan pengaruh gender dalam perbedaan performa akademik mahasiswa, yaitu ketika di groupby dan std terdapat NaN di mata kuliah kalkulus lanjut dengan gender wanita.

Informasi lain yang kami dapatkan yaitu pada tabel mata kuliah terdapat 9 mata kuliah yaitu Kalkulus dasar, Statistika Elementer, Matematika Dasar, Statistika Matematika, Operation Research, Pendahuluan Ilmu Komputer, Aljabar Abstrak, Aljabar linear, dan Kalkulus Lanjut, tetapi ternyata terdapat 5 mata kuliah yang sama sekali tidak diikuti oleh mahasiswa (tidak ada mahasiswa/mahasiswi yang mengikuti mata kuliah tsb), seperti Statistika Elementer, Matematika dasar, Operation Research, aljabar abstrak, aljabar linear.